In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-classification-nlp/SMS_train.csv
/kaggle/input/email-classification-nlp/SMS_test.csv


In [2]:
import pandas as pd
import re
import spacy 
df_test=pd.read_csv('/kaggle/input/email-classification-nlp/SMS_test.csv',encoding='latin-1')
df_train=pd.read_csv('/kaggle/input/email-classification-nlp/SMS_train.csv',encoding='latin-1')
df_train.head(10)

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
5,6,REMINDER FROM O2: To get 2.50 pounds free call...,Spam
6,7,Huh y lei...,Non-Spam
7,8,Why don't you wait 'til at least wednesday to ...,Non-Spam
8,9,Ard 6 like dat lor.,Non-Spam
9,10,Ok lor... Sony ericsson salesman... I ask shuh...,Non-Spam


In [3]:
df_test=df_test.drop(['S. No.'],axis=1)
df_train=df_train.drop(['S. No.'],axis=1)

In [4]:
def find_non_ascii(text):
    return ''.join([char for char in text if ord(char) > 127])
df_train["weird_chars"] = df_train["Message_body"].apply(find_non_ascii)
df_train[df_train["weird_chars"] != ""]


,Message_body,Label,weird_chars
3,Will ü b going to esplanade fr home?,Non-Spam,ü
4,This is the 2nd time we have tried 2 contact u...,Spam,£
30,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,Spam,££
46,You are awarded a SiPix Digital Camera! call 0...,Spam,£
59,"Its reassuring, in this crazy world.",Non-Spam,
...,...,...,...
931,How come i din c ü... Yup i cut my hair...,Non-Spam,ü
938,Todays Voda numbers ending with 7634 are selec...,Spam,£
939,Please call our customer service representativ...,Spam,££
950,Ü called dad oredi...,Non-Spam,Ü


In [5]:
def clean_ascii(text):
    return text.encode("ascii", "ignore").decode("ascii")

import re

def clean_ascii_extended(text):
    text = text.encode("ascii", "ignore").decode("ascii")
    text = text.lower()  
    text = re.sub(r'[^a-z\s]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text
df_train["Cleaned_Message"] = df_train["Message_body"].apply(clean_ascii_extended)
df_test["Cleaned_Message"] = df_test["Message_body"].apply(clean_ascii_extended)


In [6]:
!pip install contractions


In [7]:

from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

slang_dict = {
    "u": "you",
    "ur": "your",
    "lol": "laughing",
    "btw": "by the way",
    "idk": "I do not know",
    "imo": "in my opinion",
    "wanna": "want to",
    "gonna": "going to",
    "lei": "",
    "lor": "",
    "fr": "for",
    "b4": "before",
    # add more here
}

def clean_text(text):
    # ASCII encode/decode
    text = text.encode("ascii", "ignore").decode("ascii")
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    # Remove punctuation & special chars
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Replace multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Replace slang
    words = text.split()
    words = [slang_dict.get(word, word) for word in words]
    text = " ".join(words)
    # Remove stopwords using SpaCy
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text not in STOP_WORDS and len(token.text) > 2]
    return " ".join(tokens)

df_train['Cleaned_Message'] = df_train['Message_body'].apply(clean_text)
df_test['Cleaned_Message'] = df_test['Message_body'].apply(clean_text)


In [8]:
def spacy_lemmatize(text):
    doc = nlp(text)
    return " ".join([
        token.lemma_ for token in doc
        if token.lemma_ not in STOP_WORDS and token.is_alpha
    ])

df_train['Final_Text'] = df_train['Cleaned_Message'].apply(spacy_lemmatize)
df_test['Final_Text'] = df_test['Cleaned_Message'].apply(spacy_lemmatize)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df_train['Final_Text'].tolist()

tfidf = TfidfVectorizer(
    max_features=5000,       
    ngram_range=(1, 2),      
    stop_words='english',   
    min_df=5,                
    max_df=0.7               
)

X_train_tfidf = tfidf.fit_transform(corpus)
X_test_tfidf = tfidf.transform(df_test['Final_Text'].tolist())


In [10]:
df_train['Label'].value_counts()

Label
Non-Spam    835
Spam        122
Name: count, dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['Label'] = le.fit_transform(df_train['Label'])
df_test['Label'] = le.transform(df_test['Label'])


In [12]:
df_test = df_test.drop(['Message_body', 'Cleaned_Message'], axis=1)


In [13]:
df_train = df_train.drop(['Message_body', 'weird_chars', 'Cleaned_Message'], axis=1)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model = LogisticRegression(max_iter=1000)


model.fit(X_train_tfidf, df_train['Label'])

y_pred = model.predict(X_test_tfidf)

print(classification_report(df_test['Label'], y_pred))
print("Accuracy:", accuracy_score(df_test['Label'], y_pred))


              precision    recall  f1-score   support

           0       0.58      1.00      0.73        49
           1       1.00      0.53      0.69        76

    accuracy                           0.71       125
   macro avg       0.79      0.76      0.71       125
weighted avg       0.83      0.71      0.71       125

Accuracy: 0.712


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
model = LogisticRegression(max_iter=1000)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = tfidf.transform(df_train['Final_Text'])
y = df_train['Label']  

scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

print("Cross-validation accuracy scores:", scores)
print("Mean accuracy:", scores.mean())


Cross-validation accuracy scores: [0.9375     0.89583333 0.92146597 0.93193717 0.92146597]
Mean accuracy: 0.9216404886561955


In [17]:
from sklearn.linear_model import LogisticRegression
best_model = LogisticRegression(C=100, max_iter=1000)

best_model.fit(X_train_tfidf, df_train['Label'])

X_test = tfidf.transform(df_test['Final_Text'])
y_test = df_test['Label']

test_accuracy = best_model.score(X_test, y_test)
print("Test accuracy:", test_accuracy)


Test accuracy: 0.864


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, df_train['Label'])  

y_pred = model.predict(X_test_tfidf)

prediction_df = pd.DataFrame({
    'Final_Text': df_test['Final_Text'],
    'Actual_Label': df_test['Label'],
    'Predicted_Label': y_pred,
    'Correct': df_test['Label'] == y_pred
})

prediction_df.head(10)



,Final_Text,Actual_Label,Predicted_Label,Correct
0,upgrdcentre orange customer claim free camera ...,1,1,True
1,loan purpose homeowner tenant welcome previous...,1,0,False
2,congrats nokia video camera phone cost ppm ave...,1,1,True
3,urgent mobile number award prize guarantee lan...,1,1,True
4,contact date service enter phone fancy find la...,1,0,False
5,send logo lover join heart txt love mobno love...,1,0,False
6,free entry weekly competition text word win,1,1,True
7,date service cal box,1,0,False
8,free ringtone text poly text true tone help fr...,1,1,True
9,mth half price orange line rental late camera ...,1,1,True


In [ ]:
prediction_df.to_csv('predictions_logistic_regression.csv', index=False)
